# Generating C++ Kokkos Chemistry Solvers with JAFF

This notebook demonstrates how to use JAFF to generate C++ code that uses the Kokkos-kernels BDF solver for solving chemical reaction networks. We'll use the `react_COthin` network as an example, which contains 37 species and 287 reactions for carbon-oxygen chemistry.

## Overview

The Kokkos output plugin generates:
- **chemistry_ode.hpp**: Chemistry ODE system class with evaluate_function and evaluate_jacobian methods
- **chemistry_ode.cpp**: Main executable that sets up and runs the BDF solver
- **CMakeLists.txt**: Build configuration for Kokkos and Kokkos-kernels dependencies

The generated code is compatible with the Kokkos-kernels BDF solver for stiff ODEs.

## Step 1: Load the Chemical Network

First, let's load the react_COthin network, which is a carbon-oxygen chemistry network in KROME format.

In [1]:
from jaff.network import Network
from jaff.builder import Builder
import os

# Load the react_COthin network
network = Network('../networks/react_COthin')

print(f"Network loaded: {network.label}")
print(f"Number of species: {network.get_number_of_species()}")
print(f"Number of reactions: {len(network.reactions)}")

Welcome to JAFF: Just Another Fancy Format!
Loading network from ../networks/react_COthin
Network label = react_COthin


  3%|███▋                                                                                                                 | 11/350 [00:00<00:03, 101.29it/s]

KROME variable detected: @var:Hnuclei = get_Hnuclei(n(:))
KROME variable detected: @var: Te = Tgas*8.617343d-5
KROME variable detected: @var: invT = 1d0/Tgas
KROME variable detected: @var: lnTe = log(Te)
KROME variable detected: @var: T = Tgas
KROME format detected: @format:idx,R,R,R,P,P,P,P,Tmin,Tmax,rate
exp(-2.03914985e-6*lnte**8 + 0.000111954395*lnte**7 - 0.00263197617*lnte**6 + 0.0348255977*lnte**5 - 0.2877056*lnte**4 + 1.56315498*lnte**3 - 5.73932875*lnte**2 + 13.536556*lnte - 32.71396786)
3.92e-13*invte**0.6353
exp(-3.071135243196595e-9*lnte**9 - 1.856767039775261e-8*lnte**8 + 5.755614137575758e-7*lnte**7 + 4.989108920299513e-6*lnte**6 - 1.421502914054107e-5*lnte**5 - 0.0003212605213188796*lnte**4 - 0.002380861877349834*lnte**3 - 0.02026044731984691*lnte**2 - 0.7241125657826851*lnte - 28.61303380689232)
exp(-3.64916141e-6*lnte**8 + 0.000206723616*lnte**7 - 0.0050090561*lnte**6 + 0.0679539123*lnte**5 - 0.56851189*lnte**4 + 3.05803875*lnte**3 - 10.7532302*lnte**2 + 23.91596563*lnt

 11%|█████████████▎                                                                                                       | 40/350 [00:00<00:02, 119.05it/s]

7.20000000000000e-15
KROME format detected: @format:idx,R,R,R,P,P,P,Tmin,Tmax,rate
3.7e-14*exp(35.0*invt)
3.0e-11*sqrt(t32)*exp(-52000.0*invt)
3.36e-10/(sqrtgas*((tgas*1.0e-6)**0.7 + 1.0)*((tgas*0.001)**0.2))
6.77e-15*te**0.8779
1.43000000000000e-9
exp(2.238306228891639e-7*lnte**7 - 8.36671960467864e-6*lnte**6 + 0.0001073294010367247*lnte**5 - 0.0003105115447124016*lnte**4 - 0.004555120027032095*lnte**3 + 0.03599837721023835*lnte**2 + 0.2289800603272916*lnte - 20.06913897587003)
1.85e-23*tgas**1.8
5.81e-16/(tgas/56200.0)**(0.6657*log10(tgas/56200.0))
6.00000000000000e-10
exp(-9.36345888928611e-6*lnte**8 + 0.0004138398421504563*lnte**7 - 0.007879026154483455*lnte**6 + 0.0841077503763412*lnte**5 - 0.5416182856220388*lnte**4 + 2.045587822403071*lnte**3 - 3.898003964650152*lnte**2 + 3.400824447095291*lnte - 24.24914687731536)
5.6e-11*tgas**0.5*exp(-102124.0*invt)
1.0670825e-10*te**2.012*exp(-4.463*invte)/((0.2472*te + 1.0)**3.512)
exp(-2.631285809207e-6*lnte**8 + 0.0001068275202678*lnte**7

...

6.9e-32/tgas**0.4
1.3e-32/t32**0.38
1.3e-32/t32**1.0
1.3e-32/(8.0*t32**0.38)
1.3e-32/(8.0*t32**1.0)
4.67e-12/t32**0.6
1.23e-17*t32**2.49*exp(21845.6*invt)
9.62e-8*exp(-115786.2*invt)/t32**1.37
7.5e-12/t32**0.55
4.86e-12/t32**0.32
1.3e-10/tgas**0.64
0.00074*(1.0 + 0.062*exp(-145000.0*invt))*exp(-175000.0*invt)/tgas**1.5 + 1.41e-10/tgas**0.66
KROME variable detected: @var:u1 = 11.26d0*invTe
KROME variable detected: @var:u2 = 8.2d0*invTe
KROME variable detected: @var:u3 = 13.6*invTe
KROME format detected: @format:idx,R,R,R,P,P,P,P,Tmin,Tmax,rate
6.85e-8*u1**0.25*exp(-u1)/(u1 + 0.193)
1.88e-7*u2**0.25*(u2**0.5 + 1.0)*exp(-u2)/(u2 + 0.376)
3.59e-8*u3**0.34*exp(-u3)/(u3 + 0.073)
7.54e-10*invt**0.458 + 4.99e-11*tgas**0.405
(4.0e-10*tgas**0.00669 + 1.08e-11*tgas**0.517)*exp(-227.0*invt)
2.78e-15*exp(-tgas*1.2258e-6)/(tgas*0.0001)**0.2163 + 4.991e-15*(tgas*0.0001)**0.3794*exp(-tgas*8.9206e-7)
3.9e-16*tgas**0.213
6.08e-14*(tgas*0.0001)**1.96*exp(-170000.0*invt)
8.58e-17*tgas**0.757
3.25e-17*tgas

 71%|██████████████████████████████████████████████████████████████████████████████████▌                                 | 249/350 [00:00<00:00, 451.06it/s]

1.0e-10*exp(-4640.0*invt)
7.50000000000000e-10
1.20000000000000e-9
3.50000000000000e-10
1.0e-9*exp(-7080.0*invt)
1.60000000000000e-9
7.50000000000000e-10
7.0e-10*exp(-10560.0*invt)
2.50000000000000e-10
2.50000000000000e-10
4.80000000000000e-10
1.69000000000000e-9
1.50000000000000e-9
7.98e-10*exp(-1.41*invt)/t32**0.156
3.42e-10*exp(-1.41*invt)/t32**0.156
2.28000000000000e-8
1.52e-9*(2.62185*(300.0*invt)**0.5 + 0.62)
2.28000000000000e-8
9.15e-10*(2.62185*(300.0*invt)**0.5 + 0.62)
1.01000000000000e-9
6.40000000000000e-10
2.55000000000000e-8
1.73e-9*(2.578947*(300.0*invt)**0.5 + 0.62)
1.80000000000000e-9
5.03000000000000e-9
3.4093e-10*(2.578947*(300.0*invt)**0.5 + 0.62)
2.40000000000000e-10
1.00000000000000e-11
3.42000000000000e-10
4.53000000000000e-10
9.10000000000000e-10
5.20000000000000e-11
5.20000000000000e-11
2.47000000000000e-9
1.88055e-9*(1.79558*invt + 0.02427*(300.0*invt)**0.5 + 1.0)
1.45000000000000e-10
1.08256e-10*(1.79558*invt + 0.02427*(300.0*invt)**0.5 + 1.0)
1.00000000000000

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 350/350 [00:01<00:00, 325.59it/s]

KROME variable detected: @var: HnOj = fHnOj(user_Av)
5.0e-11*hnoj
5.0e-11*hnoj
5.0e-11*hnoj
1.5e-10*hnoj
2.5e-11*hnoj
2.5e-11*hnoj
7.5e-12*hnoj
2.5e-11*hnoj
5.6e-11*exp(-3.7*av)
7.0e-10*exp(-1.8*av)
2.0e-10*exp(-3.53*av)
0.46*crate
0.5*crate
2.8*crate
5.0*crate
3.0*crate
237.0*crate
0.1*crate
0.0003*crate
0.93*crate
1020.0*crate
730.0*crate
750.0*crate
117.0*crate
510.0*crate
500.0*crate
970.0*crate
421.0*crate
1170.0*crate
KROME format detected: @format:idx,R,P,P,P,rate
0.93*crate
KROME variable detected: @var:ntot=sum(n(1:nmols))
KROME format detected: @format:idx,R,R,P,Tmin,Tmax,rate
5.99e-33*ntot/(tgas/5000.0)**1.6
5.99e-33*ntot*exp(5255.0/tgas)/(tgas/5000.0)**0.64
6.16e-29*ntot/(tgas/300.0)**3.08
2.14e-29*ntot*exp(2114.0/tgas)/(tgas/300.0)**3.08
6.16e-27*ntot/(tgas/300.0)**3.08
2.14e-27*ntot*exp(2114.0/tgas)/(tgas/300.0)**3.08
6.16e-27*ntot/(tgas/300.0)**3.08
2.14e-27*ntot*exp(2114.0/tgas)/(tgas/300.0)**3.08
4.33e-32*ntot/t32
2.56e-31*ntot/t32**2
9.2e-34*ntot/t32
Variables found: 

## Step 2: Examine Network Properties

Let's examine some properties of the network to understand what we're working with.

In [2]:
# Show the first few species
print("First 10 species:")
for i, species in enumerate(network.species[:10]):
    print(f"  {i}: {species.name} (mass: {species.mass:.2f}, charge: {species.charge})")

if len(network.species) > 10:
    print(f"  ... and {len(network.species) - 10} more species")

print(f"\nTotal species: {len(network.species)}")

First 10 species:
  0: H (mass: 0.00, charge: 0)
  1: e- (mass: 0.00, charge: -1)
  2: H+ (mass: 0.00, charge: 1)
  3: He (mass: 0.00, charge: 0)
  4: He+ (mass: 0.00, charge: 1)
  5: He++ (mass: 0.00, charge: 2)
  6: H2 (mass: 0.00, charge: 0)
  7: H2+ (mass: 0.00, charge: 1)
  8: H- (mass: 0.00, charge: -1)
  9: C+ (mass: 0.00, charge: 1)
  ... and 27 more species

Total species: 37


In [3]:
# Show a few example reactions
print("First 5 reactions:")
for i, reaction in enumerate(network.reactions[:5]):
    reactants = " + ".join([f"{r.name}" for r in reaction.reactants])
    products = " + ".join([f"{p.name}" for p in reaction.products])
    print(f"  {i+1}: {reactants} -> {products}")
    print(f"      Rate: {reaction.rate}")
    print()

First 5 reactions:
  1: H + e- -> H+ + e- + e-
      Rate: 5.90824386372651e-70*exp(-2.03914985e-6*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**8 + 0.000111954395*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**7 - 0.00263197617*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**6 + 0.0348255977*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**5 - 0.2877056*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**4 + 1.56315498*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**3 - 5.73932875*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**2)*Max(2.73, Min(100000000.0, tgas))**13.536556

  2: H+ + e- -> H
      Rate: 1.49810881307214e-10/Max(2.73, Min(5500.0, tgas))**0.6353

  3: H+ + e- -> H
      Rate: 3.286733702438273e-10*exp(-3.071135243196595e-9*log(8.617343e-5*Max(5500.0, Min(100000000.0, tgas)))**9 - 1.856767039775261e-8*log(8.617343e-5*Max(5500.0, Min(100000000.0, tgas)))**8 + 5.755614137575758e-7*log(8.617343e-5*Max(5500.0, Min(100000000.0, tgas)))**7 + 4.98910

## Step 3: Generate C++ Kokkos Code

Now let's use the Builder to generate C++ code using the `kokkos_ode` template.

In [4]:
# Create builder and generate Kokkos C++ code
builder = Builder(network)
builder.build(template="kokkos_ode")

print("\nC++ Kokkos code generated successfully!")
print("Generated files are located in the builds directory.")

Building network with template: kokkos_ode
Copying /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/CMakeLists.txt to /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/builds
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/chemistry_ode.hpp -> /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/builds/chemistry_ode.hpp
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/chemistry_ode.cpp -> /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/builds/chemistry_ode.cpp
Copying /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/chemistry_ode.hpp to /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/builds
Copying /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/chemistry_ode.cpp to /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/builds
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_od

## Step 4: Examine Generated Files

Let's look at the structure and content of the generated files.

In [5]:
# List generated files
builds_dir = "../src/jaff/builds"
generated_files = [f for f in os.listdir(builds_dir) if f.endswith(('.cpp', '.hpp', '.txt'))]

print("Generated files:")
for file in sorted(generated_files):
    filepath = os.path.join(builds_dir, file)
    size = os.path.getsize(filepath)
    print(f"  {file:20} ({size:,} bytes)")

Generated files:
  CMakeLists.txt       (1,309 bytes)
  chemistry_ode.cpp    (5,911 bytes)
  chemistry_ode.hpp    (2,478 bytes)


## Step 5: Understanding the Generated Code Structure

Let's examine key components of the generated chemistry solver.

In [6]:
# Show the ChemistryODE class definition
with open("../src/jaff/builds/chemistry_ode.hpp", 'r') as f:
    lines = f.readlines()

# Find the class definition
class_start = None
for i, line in enumerate(lines):
    if "struct ChemistryODE" in line:
        class_start = i
        break

if class_start:
    print("ChemistryODE class structure:")
    print("=" * 50)
    # Show class definition and methods
    brace_count = 0
    for i in range(class_start, min(class_start + 20, len(lines))):
        line = lines[i].rstrip()
        print(f"{i+1:2d}: {line}")
        
        if '{' in line:
            brace_count += line.count('{')
        if '}' in line:
            brace_count -= line.count('}')
            if brace_count == 0:
                break

ChemistryODE class structure:
10: struct ChemistryODE {
11:     // Number of species in the chemical network
12:     // PREPROCESS_NUM_SPECIES
13:     static constexpr int neqs = 0;
14:     // PREPROCESS_END
15: 
16:     // Species indices
17:     // PREPROCESS_COMMONS
18: 
19:     // PREPROCESS_END
20: 
21:     ChemistryODE() {}
22: 
23:     // Evaluate the ODE right-hand side: dy/dt = f(t, y)
24:     template <class vec_type1, class vec_type2>
25:     KOKKOS_FUNCTION void evaluate_function(const double t, const double dt,
26:                                           const vec_type1& y, const vec_type2& f) const {
27:         // Temperature (could be passed as parameter or computed from energy)
28:         const double T = 300.0; // Default temperature in K, should be parameterized
29: 


In [7]:
# Show main function structure from chemistry_ode.cpp
with open("../src/jaff/builds/chemistry_ode.cpp", 'r') as f:
    lines = f.readlines()

# Find main function
main_start = None
for i, line in enumerate(lines):
    if "int main(" in line:
        main_start = i
        break

if main_start:
    print("Main function structure (first 25 lines):")
    print("=" * 50)
    for i in range(main_start, min(main_start + 25, len(lines))):
        line = lines[i].rstrip()
        print(f"{i+1:2d}: {line}")

Main function structure (first 25 lines):
38: int main(int argc, char* argv[]) {
39:     Kokkos::initialize(argc, argv);
40: 
41:     {
42:         using execution_space = Kokkos::DefaultExecutionSpace;
43:         using scalar_type = double;
44:         using vec_type = Kokkos::View<scalar_type*, execution_space>;
45:         using mat_type = Kokkos::View<scalar_type**, execution_space>;
46: 
47:         ChemistryODE mySys{};
48: 
49:         // Time integration parameters
50:         const scalar_type t_start = 0.0;
51:         scalar_type t_end = 3.15576e9; // 100 years in seconds default
52: 
53:         // Parse command line arguments
54:         if (argc > 1) {
55:             t_end = std::stod(argv[1]);
56:         }
57: 
58:         std::cout << "Solving Chemistry ODE System\n";
59:         std::cout << "Number of species: " << mySys.neqs << "\n";
60:         std::cout << "Number of reactions: " <<
61: // PREPROCESS_NUM_REACTIONS
62: 287


## Step 6: Building and Running Instructions

Here's how to build and run the generated C++ code:

To build and run the generated C++ chemistry solver:

1. Configure the build with CMake:
   ```
   cd ../src/jaff/builds
   cmake -S . -B build
   ```
2. Build with parallel compilation:
   ```
   cmake --build build -j
   ```
3. Run the chemistry solver:
   ```
   ./build/chemistry_ode
   ```
   Or, run with custom time interval (100 years in seconds):
   ```
   ./build/chemistry_ode 3.15576e9
   ```

The solver uses the Kokkos-kernels BDF solver which is well-suited for
stiff chemical reaction networks. It automatically adapts time steps
and uses numerical differentiation for the Jacobian.